In [1]:
import os
import csv

In [78]:
def cargar_datos_txt(fichero):
    path = os.path.join(os.getcwd(), fichero)
    lineas = csv.reader(open(path, newline='\n'))
    data = []
        
        
    for linea in lineas:
        fila = []
        for i in range(0, len(linea)):
            fila.append(int(linea[i]))
        data.append(fila)
        
    return data

In [79]:
data = cargar_datos_txt('./parametros.txt')

In [80]:
data

[[0, 1, 4], [3, 2, 5], [8, 7, 6]]

In [5]:
data
type(data)
data[0].index('8')

2

In [6]:
a = str(input('asd'))

asdw


In [44]:
def encuentra_posicion(valor, data):
    for i in range(0, len(data)):
        for j in range(0, len(data[i])): 
            
            if valor == data[i][j]:
                x, y = i,j
                break
    return x, y
            

In [45]:
i, j = encuentra_posicion(1,data)
print(i,j)

2 1


In [99]:
def crear_matriz_recompensas(data, fin):
    
    cantidad = len(data)*len(data[0])
    
    lista = []
    for i in range(0, cantidad):
        lista.append([-1]*cantidad)
    
    #Definir vecinos
    for i in range(0, len(data)):
        
        for j in range(0, len(data[i])):
            actual = data[i][j]
            
        
            
            if len(data)  > i+1:
                vecino1 = data[i+1][j]
                if vecino1 == fin:
                    lista[actual][vecino1] = 100
                    print(actual, vecino1,'= 100')
                else:
                    lista[actual][vecino1] = 0
                    print(actual, vecino1, '= 0')
                    
            if 0 <= i-1:
                vecino2 = data[i-1][j]
                if vecino2 == fin:
                    lista[actual][vecino2] = 100
                    print(actual, vecino2,'= 100')
                else:
                    lista[actual][vecino2] = 0
                    print(actual, vecino2, '= 0')
                    
            if len(data[i]) > j+1:
                vecino3 = data[i][j+1]
                if vecino3 == fin:
                    lista[actual][vecino3] = 100
                    print(actual, vecino3,'= 100')
                else:
                    lista[actual][vecino3] = 0
                    print(actual, vecino3, '= 0')
                
            if 0 <= j-1:
                vecino4 = data[i][j-1]
                if vecino4 == fin:
                    lista[actual][vecino4] = 100
                    print(actual, vecino4,'= 100')
                else:
                    lista[actual][vecino4] = 0
                    print(actual, vecino4, '= 0')
            
            
    return lista
            
            
    

In [100]:
prueba = [[0, 1, 4], [3, 2, 5], [8, 7, 6]]
recompensas = crear_matriz_recompensas(prueba, 6)

0 3 = 0
0 1 = 0
1 2 = 0
1 4 = 0
1 0 = 0
4 5 = 0
4 1 = 0
3 8 = 0
3 0 = 0
3 2 = 0
2 7 = 0
2 1 = 0
2 5 = 0
2 3 = 0
5 6 = 100
5 4 = 0
5 2 = 0
8 3 = 0
8 7 = 0
7 2 = 0
7 6 = 100
7 8 = 0
6 5 = 0
6 7 = 0


In [85]:
def cargar_datos(fichero):
    
    matriz = cargar_datos_txt(fichero)
    
    inicio = int(input('Introduce el valor de la casilla de inicio: '))
    
    fin = int(input('Introduce el valor de la casilla objetivo: '))
    
    epochs = int(input('Introduce el número de episodios de entrenamiento: '))
    
    gamma = float(input('Introduce el valor del factor de aprendizaje gamma: '))
    
    epsilon = float(input('Introduce el valor del parámetro epsilon: '))
    
    alpha = float(input('Introduce el valor del parámetro alpha: '))
    
    recompensas = crear_matriz_recompensas(data, fin)
    
    return data, recompensas, inicio, fin, epochs, gamma, epsilon, alpha
    
    

    

In [86]:
cargar_datos('./parametros.txt')

Introduce el valor de la casilla de inicio: 0
Introduce el valor de la casilla objetivo: 6
Introduce el número de episodios de entrenamiento: 6
Introduce el valor del factor de aprendizaje gamma: 6
Introduce el valor del parámetro epsilon: 6
Introduce el valor del parámetro alpha: 6
3
2
5
8
7
6
[[-1, 0, -1, 0, 0, -1, -1, -1, 0], [-1, -1, 0, -1, 0, -1, -1, 0, -1], [-1, -1, -1, -1, -1, 0, -1, 0, -1], [-1, -1, 0, -1, -1, 0, -1, -1, 0], [-1, -1, -1, -1, -1, 0, 100, -1, -1], [-1, -1, -1, -1, -1, -1, 100, -1, -1], [-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1, -1, -1, -1, -1, 100, -1, -1], [-1, -1, -1, -1, -1, -1, 100, 0, -1]]


IndexError: list index out of range